# CNN Netflix

In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Normalization
%matplotlib inline
import seaborn as sns
from keras.models import Sequential 
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, InputLayer, SimpleRNN
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn import (
    linear_model, metrics, neural_network, pipeline, model_selection
)
from sklearn.impute import SimpleImputer

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import TimeSeriesSplit

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [45]:
Netflix = pd.read_csv("/Users/jackbrennan/Documents/GitHub/Stock-Predictions/Alex/Netflix_model_ready.csv")
Netflix.date = pd.to_datetime(Netflix.date)
Netflix = Netflix.set_index("date")
Netflix = Netflix.drop(["Nas_total", 'Stock_total', 'Dow_total'],axis=1) # to remove duplicated columns
pd.DataFrame(Netflix.isna().sum()).T

,Netflix. Inc,Netflix_x,Netflix Stock,Streaming media,Reed Hastings_x,Open,High,Low,Close,Volume,...,Dow_MAvg_s_Move,Dow_EMA_Move,Dow_Disparity_Move,Dow_Disparity_s_Move,Dow_RSI_Move,target_1,target_2,target_3,target_4,target_5
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
Netflix = Netflix.dropna()
Netflix = Netflix[~(Netflix.isin([np.inf, -np.inf]).any(axis=1))] 

In [47]:
Netflix = Netflix.drop(['target_1', 'target_2', 'target_4', 'target_5'], axis=1)
target_3 = Netflix["target_3"]

#splitting into training sets 
column_indices = {name: i for i, name in enumerate(Netflix.columns)}

n = len(Netflix)
X_train = Netflix[0:int(n*0.7)]
X_val = Netflix[int(n*0.7):int(n*0.9)]
X_test = Netflix[int(n*0.9):]

y_train = target_3[0:int(n*0.7)]
y_val = target_3[int(n*0.7):int(n*0.9)]
y_test = target_3[int(n*0.9):]

In [48]:
Mscaler = MinMaxScaler() # keeps binarys at zero and 1 :)

X_train = pd.DataFrame(Mscaler.fit_transform(X_train), columns = Netflix.columns)
X_val = pd.DataFrame(Mscaler.fit_transform(X_val), columns = Netflix.columns)
X_test = pd.DataFrame(Mscaler.fit_transform(X_test), columns = Netflix.columns)

In [49]:
def kbest_creator(k, df, df_val, df_test):
    """
    returns data frame of principle componets of # of k best features 
    """
    # apply SelectKBest class to extract top 40 best features
    bestfeatures = SelectKBest(score_func=f_regression, k=k)
    best_fit = bestfeatures.fit(df, y_train)
    best_scores = pd.DataFrame(best_fit.scores_)
    best_columns = pd.DataFrame(Netflix.columns)
    
    # concatenate the dataframes for better visualization
    features_score = pd.concat([best_columns, best_scores], axis=1)
    features_score.columns = ['Features', 'Score']  # naming the dataframe columns
    feats_kb = list(features_score.nlargest(k, 'Score').iloc[1:k]['Features'])

    pca = PCA().fit(df[feats_kb])
    pca_scores = np.cumsum(pca.explained_variance_ratio_) * 100
    res = next(x for x, val in enumerate(pca_scores) if val > 85)
    res = res + 1
    res
    
    pca_kb_1 = PCA(n_components = res).fit(df[feats_kb].to_numpy())
    df = pca_kb_1.transform(df[feats_kb].to_numpy())
    df_val = pca_kb_1.transform(df_val[feats_kb].to_numpy())
    df_test = pca_kb_1.transform(df_test[feats_kb].to_numpy())
    return df, df_val, df_test


In [50]:
X_train_kb_10, X_val_kb_10, X_test_kb_10 = kbest_creator(10, X_train, X_val, X_test) 
X_train_kb_25, X_val_kb_25, X_test_kb_25 = kbest_creator(25, X_train, X_val, X_test) 
X_train_kb_40, X_val_kb_40, X_test_kb_40 = kbest_creator(40, X_train, X_val, X_test) 
X_train_kb_55, X_val_kb_55, X_test_kb_55 = kbest_creator(55, X_train, X_val, X_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


# Model Testing

In [51]:
early_stopping_monitor = EarlyStopping(patience=4)

In [52]:
def df_to_X_y2(df, target, window_size=5):
  # converts to matrix of numpy arrays
  X = []
  y = []
  for i in range(len(df)-window_size): # length of data frame - window_size so it does't take empty values at the end, 
    # does force you to loose the last 5 values, could fix with padding
    row = [r for r in df[i:i+window_size]] # grabs row i and all rows above within the window size length
    X.append(row) # creates 3 dimentional array, (# obseravtions, # rows in window, # features)
    label = target[i+window_size] # pulls the target variable after the window, target varible needs to be column zero in this 
    y.append(label) # returns (N,) martix of targets i+window_length time periods away
  return np.array(X), np.array(y)

In [53]:
# converting to window format, in this case 5 periods
X_train_kb_40, train_5w = df_to_X_y2(X_train_kb_40,y_train)
X_val_kb_40, val_5w = df_to_X_y2(X_val_kb_40, y_val)
X_test_kb_40, test_5w = df_to_X_y2(X_test_kb_40,y_test) 

X_train_kb_10, _ = df_to_X_y2(X_train_kb_10,y_train)
X_val_kb_10, _ = df_to_X_y2(X_val_kb_10, y_val)
X_test_kb_10, _ = df_to_X_y2(X_test_kb_10,y_test) 

X_train_kb_25, _ = df_to_X_y2(X_train_kb_25,y_train)
X_val_kb_25, _ = df_to_X_y2(X_val_kb_25, y_val)
X_test_kb_25, _ = df_to_X_y2(X_test_kb_25,y_test) 

X_train_kb_55, _ = df_to_X_y2(X_train_kb_55,y_train)
X_val_kb_55, _ = df_to_X_y2(X_val_kb_55, y_val)
X_test_kb_55, _ = df_to_X_y2(X_test_kb_55,y_test) 

### Model Format 1

In [54]:
# Model 1_1 - 1_4, using differnt k best pca variables
n_steps = X_train_kb_10.shape[1]
n_features = X_train_kb_10.shape[2]

model1_1 = Sequential()
model1_1.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_1.add(Flatten())
model1_1.add(Dense(25, activation='relu')) 
model1_1.add(Dense(1, activation='sigmoid'))

model1_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_1.fit(X_train_kb_10, train_5w,epochs=30,  validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 13ms/step - loss: 0.6942 - binary_accuracy: 0.4850 - val_loss: 0.6992 - val_binary_accuracy: 0.4460
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6872 - binary_accuracy: 0.5711 - val_loss: 0.6913 - val_binary_accuracy: 0.5612
Epoch 3/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6845 - binary_accuracy: 0.5812 - val_loss: 0.6936 - val_binary_accuracy: 0.5180
Epoch 4/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6826 - binary_accuracy: 0.5611 - val_loss: 0.6955 - val_binary_accuracy: 0.5036
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6806 - binary_accuracy: 0.5852 - val_loss: 0.6977 - val_binary_accuracy: 0.5108
Epoch 6/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6794 - binary_accuracy: 0.5711 - val_loss: 0.7031 - val_binary_accuracy: 0.5036


In [71]:
y_hat1_1 = model1_1.predict(X_test_kb_10)
y_hat1_1 = y_hat1_1 > .5
metrics.accuracy_score(test_5w,y_hat1_1)

0.4411764705882353

In [55]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model1_2 = Sequential()
model1_2.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_2.add(Flatten())
model1_2.add(Dense(25, activation='relu')) 
model1_2.add(Dense(1, activation='sigmoid'))

model1_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 12ms/step - loss: 0.6940 - binary_accuracy: 0.5010 - val_loss: 0.6966 - val_binary_accuracy: 0.4892
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6813 - binary_accuracy: 0.5832 - val_loss: 0.7033 - val_binary_accuracy: 0.4532
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6660 - binary_accuracy: 0.6253 - val_loss: 0.6989 - val_binary_accuracy: 0.4892
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6577 - binary_accuracy: 0.6293 - val_loss: 0.7042 - val_binary_accuracy: 0.5108
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6497 - binary_accuracy: 0.6513 - val_loss: 0.7076 - val_binary_accuracy: 0.5180


In [72]:
y_hat1_2 = model1_2.predict(X_test_kb_25)
y_hat1_2 = y_hat1_2 > .5
metrics.accuracy_score(test_5w,y_hat1_2)

0.4264705882352941

In [56]:
n_steps = X_train_kb_40.shape[1]
n_features = X_train_kb_40.shape[2]

model1_3 = Sequential()
model1_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_3.add(Flatten())
model1_3.add(Dense(25, activation='relu')) 
model1_3.add(Dense(1, activation='sigmoid'))

model1_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_3.fit(X_train_kb_40, train_5w,epochs=30,  validation_data=(X_val_kb_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 14ms/step - loss: 0.6966 - binary_accuracy: 0.5070 - val_loss: 0.6929 - val_binary_accuracy: 0.4892
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6776 - binary_accuracy: 0.6253 - val_loss: 0.6946 - val_binary_accuracy: 0.4892
Epoch 3/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6658 - binary_accuracy: 0.6313 - val_loss: 0.7003 - val_binary_accuracy: 0.4892
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6551 - binary_accuracy: 0.6573 - val_loss: 0.7032 - val_binary_accuracy: 0.4676
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6450 - binary_accuracy: 0.6633 - val_loss: 0.7090 - val_binary_accuracy: 0.4676


In [73]:
y_hat1_3 = model1_3.predict(X_test_kb_40)
y_hat1_3 = y_hat1_3 > .5
metrics.accuracy_score(test_5w,y_hat1_3)

0.39705882352941174

In [57]:
n_steps = X_train_kb_55.shape[1]
n_features = X_train_kb_55.shape[2]

model1_4 = Sequential()
model1_4.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_4.add(Flatten())
model1_4.add(Dense(25, activation='relu')) 
model1_4.add(Dense(1, activation='sigmoid'))

model1_4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_4.fit(X_train_kb_55, train_5w,epochs=30,  validation_data=(X_val_kb_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 12ms/step - loss: 0.7046 - binary_accuracy: 0.4709 - val_loss: 0.6900 - val_binary_accuracy: 0.5108
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6801 - binary_accuracy: 0.6132 - val_loss: 0.7016 - val_binary_accuracy: 0.4820
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6674 - binary_accuracy: 0.6613 - val_loss: 0.7029 - val_binary_accuracy: 0.5036
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6552 - binary_accuracy: 0.6653 - val_loss: 0.7046 - val_binary_accuracy: 0.5036
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6425 - binary_accuracy: 0.6914 - val_loss: 0.7117 - val_binary_accuracy: 0.4820


In [74]:
y_hat1_4 = model1_4.predict(X_test_kb_55)
y_hat1_4 = y_hat1_4 > .5
metrics.accuracy_score(test_5w,y_hat1_4)

0.45588235294117646

### Model 2, Less Filters

In [64]:
# Model 2_1 - 2_4, using differnt k best pca variables. reduced filters
n_steps = X_train_kb_10.shape[1]
n_features = X_train_kb_10.shape[2]

model2_1 = Sequential()
model2_1.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_1.add(Flatten())
model2_1.add(Dense(25, activation='relu')) 
model2_1.add(Dense(1, activation='sigmoid'))

model2_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_1.fit(X_train_kb_10, train_5w,epochs=30,  validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 15ms/step - loss: 0.6946 - binary_accuracy: 0.4890 - val_loss: 0.6909 - val_binary_accuracy: 0.4964
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6926 - binary_accuracy: 0.5190 - val_loss: 0.6891 - val_binary_accuracy: 0.4892
Epoch 3/30
16/16 [==============================] - 0s 15ms/step - loss: 0.6914 - binary_accuracy: 0.5331 - val_loss: 0.6889 - val_binary_accuracy: 0.5036
Epoch 4/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6904 - binary_accuracy: 0.5411 - val_loss: 0.6894 - val_binary_accuracy: 0.4964
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6896 - binary_accuracy: 0.5551 - val_loss: 0.6889 - val_binary_accuracy: 0.5036
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6886 - binary_accuracy: 0.5631 - val_loss: 0.6888 - val_binary_accuracy: 0.4892
Epoch 7/30
16/16 [==============================] - 0s 7ms/step - loss: 0.

In [75]:
y_hat2_1 = model2_1.predict(X_test_kb_10)
y_hat2_1 = y_hat2_1 > .5
metrics.accuracy_score(test_5w,y_hat2_1)

0.4264705882352941

In [63]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model2_2 = Sequential()
model2_2.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_2.add(Flatten())
model2_2.add(Dense(25, activation='relu')) 
model2_2.add(Dense(1, activation='sigmoid'))

model2_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 43ms/step - loss: 0.7065 - binary_accuracy: 0.4709 - val_loss: 0.6994 - val_binary_accuracy: 0.5540
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.7004 - binary_accuracy: 0.4950 - val_loss: 0.6966 - val_binary_accuracy: 0.5683
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6959 - binary_accuracy: 0.5050 - val_loss: 0.6978 - val_binary_accuracy: 0.5683
Epoch 4/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6920 - binary_accuracy: 0.5331 - val_loss: 0.6974 - val_binary_accuracy: 0.5755
Epoch 5/30
16/16 [==============================] - 0s 20ms/step - loss: 0.6888 - binary_accuracy: 0.5371 - val_loss: 0.6980 - val_binary_accuracy: 0.5612
Epoch 6/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6858 - binary_accuracy: 0.5591 - val_loss: 0.6993 - val_binary_accuracy: 0.5180


In [76]:
y_hat2_2 = model2_2.predict(X_test_kb_25)
y_hat2_2 = y_hat2_2 > .5
metrics.accuracy_score(test_5w,y_hat2_2)

0.5

In [62]:
n_steps = X_train_kb_40.shape[1]
n_features = X_train_kb_40.shape[2]

model2_3 = Sequential()
model2_3.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_3.add(Flatten())
model2_3.add(Dense(25, activation='relu')) 
model2_3.add(Dense(1, activation='sigmoid'))

model2_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_3.fit(X_train_kb_40, train_5w,epochs=30,  validation_data=(X_val_kb_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 20ms/step - loss: 0.6984 - binary_accuracy: 0.4790 - val_loss: 0.7075 - val_binary_accuracy: 0.4964
Epoch 2/30
16/16 [==============================] - 0s 13ms/step - loss: 0.6898 - binary_accuracy: 0.5090 - val_loss: 0.7044 - val_binary_accuracy: 0.4892
Epoch 3/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6841 - binary_accuracy: 0.5411 - val_loss: 0.7023 - val_binary_accuracy: 0.4820
Epoch 4/30
16/16 [==============================] - 0s 10ms/step - loss: 0.6794 - binary_accuracy: 0.5551 - val_loss: 0.7044 - val_binary_accuracy: 0.4460
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6758 - binary_accuracy: 0.5671 - val_loss: 0.7037 - val_binary_accuracy: 0.4604
Epoch 6/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6725 - binary_accuracy: 0.5852 - val_loss: 0.7070 - val_binary_accuracy: 0.4604
Epoch 7/30
16/16 [==============================] - 0s 8ms/step - loss: 

In [79]:
y_hat2_3 = model2_3.predict(X_test_kb_40)
y_hat2_3 = y_hat2_3 > .5
metrics.accuracy_score(test_5w,y_hat2_3)

0.5

In [61]:
n_steps = X_train_kb_55.shape[1]
n_features = X_train_kb_55.shape[2]

model2_4 = Sequential()
model2_4.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_4.add(Flatten())
model2_4.add(Dense(25, activation='relu')) 
model2_4.add(Dense(1, activation='sigmoid'))

model2_4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_4.fit(X_train_kb_55, train_5w,epochs=30,  validation_data=(X_val_kb_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 12ms/step - loss: 0.6980 - binary_accuracy: 0.4950 - val_loss: 0.6924 - val_binary_accuracy: 0.5180
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6924 - binary_accuracy: 0.5150 - val_loss: 0.6876 - val_binary_accuracy: 0.5108
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6867 - binary_accuracy: 0.5411 - val_loss: 0.6885 - val_binary_accuracy: 0.4964
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6837 - binary_accuracy: 0.5691 - val_loss: 0.6916 - val_binary_accuracy: 0.5252
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6798 - binary_accuracy: 0.5752 - val_loss: 0.6931 - val_binary_accuracy: 0.5324
Epoch 6/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6765 - binary_accuracy: 0.5872 - val_loss: 0.6907 - val_binary_accuracy: 0.5180


In [78]:
y_hat2_4 = model2_4.predict(X_test_kb_55)
y_hat2_4 = y_hat2_4 > .5
metrics.accuracy_score(test_5w,y_hat2_4)

0.39705882352941174

### Model 3, Adding layers

In [65]:
# Model 3_1 - 3_4, adding layers
n_steps = X_train_kb_10.shape[1]
n_features = X_train_kb_10.shape[2]

model3_1 = Sequential()
model3_1.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_1.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_1.add(MaxPooling1D(pool_size=2))
model3_1.add(Flatten())
model3_1.add(Dense(8, activation='relu')) 
model3_1.add(Dense(1, activation='sigmoid'))

model3_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_1.fit(X_train_kb_10, train_5w,epochs=30,  validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 28ms/step - loss: 0.6942 - binary_accuracy: 0.4930 - val_loss: 0.6946 - val_binary_accuracy: 0.4532
Epoch 2/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6934 - binary_accuracy: 0.5090 - val_loss: 0.6936 - val_binary_accuracy: 0.4604
Epoch 3/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6928 - binary_accuracy: 0.5170 - val_loss: 0.6928 - val_binary_accuracy: 0.4892
Epoch 4/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6923 - binary_accuracy: 0.5251 - val_loss: 0.6924 - val_binary_accuracy: 0.4676
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6920 - binary_accuracy: 0.5271 - val_loss: 0.6918 - val_binary_accuracy: 0.5252
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6918 - binary_accuracy: 0.5331 - val_loss: 0.6916 - val_binary_accuracy: 0.5540
Epoch 7/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6916 - binar

In [80]:
y_hat3_1 = model3_1.predict(X_test_kb_10)
y_hat3_1 = y_hat3_1 > .5
metrics.accuracy_score(test_5w,y_hat3_1)

0.4117647058823529

In [66]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model3_2 = Sequential()
model3_2.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_2.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_2.add(MaxPooling1D(pool_size=2))
model3_2.add(Flatten())
model3_2.add(Dense(25, activation='relu')) 
model3_2.add(Dense(1, activation='sigmoid'))

model3_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 16ms/step - loss: 0.6935 - binary_accuracy: 0.5150 - val_loss: 0.6878 - val_binary_accuracy: 0.5827
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6905 - binary_accuracy: 0.5150 - val_loss: 0.6875 - val_binary_accuracy: 0.5827
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6886 - binary_accuracy: 0.5251 - val_loss: 0.6881 - val_binary_accuracy: 0.6043
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6866 - binary_accuracy: 0.5351 - val_loss: 0.6880 - val_binary_accuracy: 0.5827
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6842 - binary_accuracy: 0.5471 - val_loss: 0.6882 - val_binary_accuracy: 0.5612
Epoch 6/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6820 - binary_accuracy: 0.5471 - val_loss: 0.6886 - val_binary_accuracy: 0.5324


In [81]:
y_hat3_2 = model3_2.predict(X_test_kb_25)
y_hat3_2 = y_hat3_2 > .5
metrics.accuracy_score(test_5w,y_hat3_2)

0.3235294117647059

In [67]:
n_steps = X_train_kb_40.shape[1]
n_features = X_train_kb_40.shape[2]

model3_3 = Sequential()
model3_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_3.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_3.add(MaxPooling1D(pool_size=2))
model3_3.add(Flatten())
model3_3.add(Dense(25, activation='relu')) 
model3_3.add(Dense(1, activation='sigmoid'))

model3_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_3.fit(X_train_kb_40, train_5w,epochs=30,  validation_data=(X_val_kb_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 33ms/step - loss: 0.6924 - binary_accuracy: 0.5110 - val_loss: 0.6911 - val_binary_accuracy: 0.5540
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6902 - binary_accuracy: 0.5451 - val_loss: 0.6909 - val_binary_accuracy: 0.5612
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6888 - binary_accuracy: 0.5291 - val_loss: 0.6899 - val_binary_accuracy: 0.5683
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6866 - binary_accuracy: 0.5631 - val_loss: 0.6905 - val_binary_accuracy: 0.5324
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6848 - binary_accuracy: 0.5511 - val_loss: 0.6892 - val_binary_accuracy: 0.5396
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6821 - binary_accuracy: 0.5752 - val_loss: 0.6898 - val_binary_accuracy: 0.5180
Epoch 7/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6

In [82]:
y_hat3_3 = model3_3.predict(X_test_kb_40)
y_hat3_3 = y_hat3_3 > .5
metrics.accuracy_score(test_5w,y_hat3_3)

0.39705882352941174

In [68]:
n_steps = X_train_kb_55.shape[1]
n_features = X_train_kb_55.shape[2]

model3_4 = Sequential()
model3_4.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_4.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_4.add(MaxPooling1D(pool_size=2))
model3_4.add(Flatten())
model3_4.add(Dense(25, activation='relu')) 
model3_4.add(Dense(1, activation='sigmoid'))

model3_4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_4.fit(X_train_kb_55, train_5w,epochs=30,  validation_data=(X_val_kb_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 14ms/step - loss: 0.6965 - binary_accuracy: 0.5030 - val_loss: 0.6925 - val_binary_accuracy: 0.5324
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6927 - binary_accuracy: 0.5311 - val_loss: 0.6922 - val_binary_accuracy: 0.5755
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6916 - binary_accuracy: 0.5431 - val_loss: 0.6900 - val_binary_accuracy: 0.6043
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6899 - binary_accuracy: 0.5531 - val_loss: 0.6898 - val_binary_accuracy: 0.6259
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6889 - binary_accuracy: 0.5551 - val_loss: 0.6901 - val_binary_accuracy: 0.6043
Epoch 6/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6876 - binary_accuracy: 0.5431 - val_loss: 0.6894 - val_binary_accuracy: 0.5971
Epoch 7/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6

In [83]:
y_hat3_4 = model3_4.predict(X_test_kb_55)
y_hat3_4 = y_hat3_4 > .5
metrics.accuracy_score(test_5w,y_hat3_4)

0.5588235294117647

### Model 4, adding conv layer after pooling

In [86]:
# Model 4_1 - 4_4, adding conv layer after pooling
n_steps = X_train_kb_10.shape[1]
n_features = X_train_kb_10.shape[2]

model4_1 = Sequential()
model4_1.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_1.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model4_1.add(MaxPooling1D(pool_size=2))
model4_1.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model4_1.add(Flatten())
model4_1.add(Dense(8, activation='relu')) 
model4_1.add(Dense(1, activation='sigmoid'))

model4_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_1.fit(X_train_kb_10, train_5w,epochs=30,  validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 30ms/step - loss: 0.6949 - binary_accuracy: 0.5130 - val_loss: 0.6943 - val_binary_accuracy: 0.5252
Epoch 2/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6910 - binary_accuracy: 0.5170 - val_loss: 0.6874 - val_binary_accuracy: 0.5971
Epoch 3/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6904 - binary_accuracy: 0.5331 - val_loss: 0.6857 - val_binary_accuracy: 0.5971
Epoch 4/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6885 - binary_accuracy: 0.5291 - val_loss: 0.6855 - val_binary_accuracy: 0.5827
Epoch 5/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6882 - binary_accuracy: 0.5271 - val_loss: 0.6879 - val_binary_accuracy: 0.5612
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6863 - binary_accuracy: 0.5611 - val_loss: 0.6847 - val_binary_accuracy: 0.5971
Epoch 7/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6855 - binar

In [87]:
y_hat4_1 = model4_1.predict(X_test_kb_10)
y_hat4_1 = y_hat4_1 > .5
metrics.accuracy_score(test_5w,y_hat4_1)

0.45588235294117646

In [88]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model4_2 = Sequential()
model4_2.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_2.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model4_2.add(MaxPooling1D(pool_size=2))
model4_2.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model4_2.add(Flatten())
model4_2.add(Dense(25, activation='relu')) 
model4_2.add(Dense(1, activation='sigmoid'))

model4_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 17ms/step - loss: 0.6938 - binary_accuracy: 0.4830 - val_loss: 0.6926 - val_binary_accuracy: 0.5252
Epoch 2/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6915 - binary_accuracy: 0.5190 - val_loss: 0.6924 - val_binary_accuracy: 0.5324
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6901 - binary_accuracy: 0.5431 - val_loss: 0.6922 - val_binary_accuracy: 0.5324
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6880 - binary_accuracy: 0.5511 - val_loss: 0.6911 - val_binary_accuracy: 0.5755
Epoch 5/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6867 - binary_accuracy: 0.5291 - val_loss: 0.6902 - val_binary_accuracy: 0.5827
Epoch 6/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6830 - binary_accuracy: 0.5351 - val_loss: 0.6922 - val_binary_accuracy: 0.5396
Epoch 7/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6791 - binar

In [91]:
y_hat4_2 = model4_2.predict(X_test_kb_25)
y_hat4_2 = y_hat4_2 > .5
metrics.accuracy_score(test_5w,y_hat4_2)

0.4117647058823529

In [89]:
n_steps = X_train_kb_40.shape[1]
n_features = X_train_kb_40.shape[2]

model4_3 = Sequential()
model4_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_3.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model4_3.add(MaxPooling1D(pool_size=2))
model4_3.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model4_3.add(Flatten())
model4_3.add(Dense(25, activation='relu')) 
model4_3.add(Dense(1, activation='sigmoid'))

model4_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_3.fit(X_train_kb_40, train_5w,epochs=30,  validation_data=(X_val_kb_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 20ms/step - loss: 0.6938 - binary_accuracy: 0.5150 - val_loss: 0.6894 - val_binary_accuracy: 0.5827
Epoch 2/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6913 - binary_accuracy: 0.5170 - val_loss: 0.6903 - val_binary_accuracy: 0.5827
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6898 - binary_accuracy: 0.5170 - val_loss: 0.6895 - val_binary_accuracy: 0.5827
Epoch 4/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6880 - binary_accuracy: 0.5251 - val_loss: 0.6884 - val_binary_accuracy: 0.5827
Epoch 5/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6857 - binary_accuracy: 0.5331 - val_loss: 0.6875 - val_binary_accuracy: 0.5899
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6830 - binary_accuracy: 0.5651 - val_loss: 0.6881 - val_binary_accuracy: 0.5683
Epoch 7/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6

In [92]:
y_hat4_3 = model4_3.predict(X_test_kb_40)
y_hat4_3 = y_hat4_3 > .5
metrics.accuracy_score(test_5w,y_hat4_3)

0.4411764705882353

In [94]:
n_steps = X_train_kb_55.shape[1]
n_features = X_train_kb_55.shape[2]

model4_4 = Sequential()
model4_4.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_4.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model4_4.add(MaxPooling1D(pool_size=2))
model4_4.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model4_4.add(Flatten())
model4_4.add(Dense(25, activation='relu')) 
model4_4.add(Dense(1, activation='sigmoid'))

model4_4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_4.fit(X_train_kb_55, train_5w,epochs=30,  validation_data=(X_val_kb_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 18ms/step - loss: 0.6979 - binary_accuracy: 0.4770 - val_loss: 0.6931 - val_binary_accuracy: 0.5540
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6920 - binary_accuracy: 0.5471 - val_loss: 0.6874 - val_binary_accuracy: 0.5971
Epoch 3/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6901 - binary_accuracy: 0.5391 - val_loss: 0.6856 - val_binary_accuracy: 0.5755
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6887 - binary_accuracy: 0.5451 - val_loss: 0.6839 - val_binary_accuracy: 0.5971
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6875 - binary_accuracy: 0.5491 - val_loss: 0.6832 - val_binary_accuracy: 0.6187
Epoch 6/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6859 - binary_accuracy: 0.5371 - val_loss: 0.6851 - val_binary_accuracy: 0.5683
Epoch 7/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6835 - binar

In [95]:
y_hat4_4 = model4_4.predict(X_test_kb_55)
y_hat4_4 = y_hat4_4 > .5
metrics.accuracy_score(test_5w,y_hat4_4)

0.45588235294117646

# Trying the Correlation Method

In [100]:
corr_matrix = X_train.corr()
corr_matrix

feats_corr_10 = list(pd.DataFrame(corr_matrix['target_3'].sort_values(ascending=False).iloc[1:11]).reset_index()['index'])
feats_corr_25 = list(pd.DataFrame(corr_matrix['target_3'].sort_values(ascending=False).iloc[1:26]).reset_index()['index'])
feats_corr_40 = list(pd.DataFrame(corr_matrix['target_3'].sort_values(ascending=False).iloc[1:41]).reset_index()['index'])
feats_corr_55 = list(pd.DataFrame(corr_matrix['target_3'].sort_values(ascending=False).iloc[1:56]).reset_index()['index'])

X_train_cr_10 = X_train[feats_corr_10]
X_test_cr_10 = X_test[feats_corr_10]
X_val_cr_10 = X_val[feats_corr_10]
X_train_cr_25 = X_train[feats_corr_25]
X_test_cr_25 = X_test[feats_corr_25]
X_val_cr_25 = X_val[feats_corr_25]
X_train_cr_40 = X_train[feats_corr_40]
X_test_cr_40 = X_test[feats_corr_40]
X_val_cr_40 = X_val[feats_corr_40]
X_train_cr_55 = X_train[feats_corr_55]
X_test_cr_55 = X_test[feats_corr_55]
X_val_cr_55 = X_val[feats_corr_55]

def pca_finder(df, df_val, df_test):
    
    pca = PCA().fit(df)
    pca_scores = np.cumsum(pca.explained_variance_ratio_) * 100
    res = next(x for x, val in enumerate(pca_scores) if val > 85)
    res = res + 1
    res
    
    pca_cr_1 = PCA(n_components = res).fit(df.to_numpy())
    df = pca_cr_1.transform(df.to_numpy())
    df_val = pca_cr_1.transform(df_val.to_numpy())
    df_test = pca_cr_1.transform(df_test.to_numpy())
    return df, df_val, df_test

X_train_cr_10, X_val_cr_10, X_test_cr_10 = pca_finder(X_train_cr_10, X_val_cr_10, X_test_cr_10)
X_train_cr_25, X_val_cr_25, X_test_cr_25 = pca_finder(X_train_cr_25, X_val_cr_25, X_test_cr_25)
X_train_cr_40, X_val_cr_40, X_test_cr_40 = pca_finder(X_train_cr_40, X_val_cr_40, X_test_cr_40)
X_train_cr_55, X_val_cr_55, X_test_cr_55 = pca_finder(X_train_cr_55, X_val_cr_55, X_test_cr_55)

In [101]:
X_train_cr_40, train_5w = df_to_X_y2(X_train_cr_40,y_train)
X_val_cr_40, val_5w = df_to_X_y2(X_val_cr_40, y_val)
X_test_cr_40, test_5w = df_to_X_y2(X_test_cr_40,y_test) 

X_train_cr_10, _ = df_to_X_y2(X_train_cr_10,y_train)
X_val_cr_10, _ = df_to_X_y2(X_val_cr_10, y_val)
X_test_cr_10, _ = df_to_X_y2(X_test_cr_10,y_test) 

X_train_cr_25, _ = df_to_X_y2(X_train_cr_25,y_train)
X_val_cr_25, _ = df_to_X_y2(X_val_cr_25, y_val)
X_test_cr_25, _ = df_to_X_y2(X_test_cr_25,y_test) 

X_train_cr_55, _ = df_to_X_y2(X_train_cr_55,y_train)
X_val_cr_55, _ = df_to_X_y2(X_val_cr_55, y_val)
X_test_cr_55, _ = df_to_X_y2(X_test_cr_55,y_test) 

In [102]:
# Model 2_1 - 2_4, using differnt k best pca variables. reduced filters
n_steps = X_train_cr_10.shape[1]
n_features = X_train_cr_10.shape[2]

model2_1_cr = Sequential()
model2_1_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_1_cr.add(Flatten())
model2_1_cr.add(Dense(25, activation='relu')) 
model2_1_cr.add(Dense(1, activation='sigmoid'))

model2_1_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_1_cr.fit(X_train_cr_10, train_5w,epochs=30,  validation_data=(X_val_cr_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 17ms/step - loss: 0.6953 - binary_accuracy: 0.5210 - val_loss: 0.6815 - val_binary_accuracy: 0.5755
Epoch 2/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6916 - binary_accuracy: 0.5331 - val_loss: 0.6812 - val_binary_accuracy: 0.5827
Epoch 3/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6884 - binary_accuracy: 0.5451 - val_loss: 0.6802 - val_binary_accuracy: 0.5899
Epoch 4/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6858 - binary_accuracy: 0.5551 - val_loss: 0.6815 - val_binary_accuracy: 0.5252
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6838 - binary_accuracy: 0.5671 - val_loss: 0.6811 - val_binary_accuracy: 0.5324
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6816 - binary_accuracy: 0.5671 - val_loss: 0.6818 - val_binary_accuracy: 0.5540
Epoch 7/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6800 - binar

In [104]:
y_hat2_1_cr = model2_1_cr.predict(X_test_cr_10)
y_hat2_1_cr = y_hat2_1_cr > .5
metrics.accuracy_score(test_5w,y_hat2_1_cr)

0.5147058823529411

In [105]:
n_steps = X_train_cr_25.shape[1]
n_features = X_train_cr_25.shape[2]

model2_2_cr = Sequential()
model2_2_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_2_cr.add(Flatten())
model2_2_cr.add(Dense(25, activation='relu')) 
model2_2_cr.add(Dense(1, activation='sigmoid'))

model2_2_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_2_cr.fit(X_train_cr_25, train_5w,epochs=30,  validation_data=(X_val_cr_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 33ms/step - loss: 0.7028 - binary_accuracy: 0.4749 - val_loss: 0.7118 - val_binary_accuracy: 0.4245
Epoch 2/30
16/16 [==============================] - 0s 17ms/step - loss: 0.6932 - binary_accuracy: 0.5251 - val_loss: 0.7049 - val_binary_accuracy: 0.4748
Epoch 3/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6889 - binary_accuracy: 0.5371 - val_loss: 0.7019 - val_binary_accuracy: 0.4964
Epoch 4/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6850 - binary_accuracy: 0.5471 - val_loss: 0.6984 - val_binary_accuracy: 0.4892
Epoch 5/30
16/16 [==============================] - 0s 10ms/step - loss: 0.6816 - binary_accuracy: 0.5511 - val_loss: 0.7001 - val_binary_accuracy: 0.4676
Epoch 6/30
16/16 [==============================] - 0s 14ms/step - loss: 0.6789 - binary_accuracy: 0.5571 - val_loss: 0.6990 - val_binary_accuracy: 0.4820
Epoch 7/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6760 - bi

In [106]:
y_hat2_2_cr = model2_2_cr.predict(X_test_cr_25)
y_hat2_2_cr = y_hat2_2_cr > .5
metrics.accuracy_score(test_5w,y_hat2_2_cr)

0.47058823529411764

In [107]:
n_steps = X_train_cr_40.shape[1]
n_features = X_train_cr_40.shape[2]

model2_3_cr = Sequential()
model2_3_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_3_cr.add(Flatten())
model2_3_cr.add(Dense(25, activation='relu')) 
model2_3_cr.add(Dense(1, activation='sigmoid'))

model2_3_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_3_cr.fit(X_train_cr_40, train_5w,epochs=30,  validation_data=(X_val_cr_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 2s 26ms/step - loss: 0.7049 - binary_accuracy: 0.4990 - val_loss: 0.7029 - val_binary_accuracy: 0.4820
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6964 - binary_accuracy: 0.5110 - val_loss: 0.6968 - val_binary_accuracy: 0.4892
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6921 - binary_accuracy: 0.5431 - val_loss: 0.6936 - val_binary_accuracy: 0.5108
Epoch 4/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6887 - binary_accuracy: 0.5671 - val_loss: 0.6931 - val_binary_accuracy: 0.5396
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6854 - binary_accuracy: 0.5792 - val_loss: 0.6941 - val_binary_accuracy: 0.5324
Epoch 6/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6818 - binary_accuracy: 0.5912 - val_loss: 0.6922 - val_binary_accuracy: 0.5540
Epoch 7/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6784 - binar

In [108]:
y_hat2_3_cr = model2_3_cr.predict(X_test_cr_40)
y_hat2_3_cr = y_hat2_3_cr > .5
metrics.accuracy_score(test_5w,y_hat2_3_cr)

0.5

In [111]:
n_steps = X_train_cr_55.shape[1]
n_features = X_train_cr_55.shape[2]

model2_4_cr = Sequential()
model2_4_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_4_cr.add(Flatten())
model2_4_cr.add(Dense(25, activation='relu')) 
model2_4_cr.add(Dense(1, activation='sigmoid'))

model2_4_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_4_cr.fit(X_train_cr_55, train_5w,epochs=30,  validation_data=(X_val_cr_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 31ms/step - loss: 0.7283 - binary_accuracy: 0.4830 - val_loss: 0.7159 - val_binary_accuracy: 0.4604
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.7066 - binary_accuracy: 0.5210 - val_loss: 0.7024 - val_binary_accuracy: 0.4964
Epoch 3/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6998 - binary_accuracy: 0.5090 - val_loss: 0.6958 - val_binary_accuracy: 0.5180
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6936 - binary_accuracy: 0.5170 - val_loss: 0.6936 - val_binary_accuracy: 0.5180
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6885 - binary_accuracy: 0.5271 - val_loss: 0.6925 - val_binary_accuracy: 0.5180
Epoch 6/30
16/16 [==============================] - 0s 20ms/step - loss: 0.6835 - binary_accuracy: 0.5411 - val_loss: 0.6917 - val_binary_accuracy: 0.5108
Epoch 7/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6798 - bina

In [112]:
y_hat2_4_cr = model2_4_cr.predict(X_test_cr_55)
y_hat2_4_cr = y_hat2_4_cr > .5
metrics.accuracy_score(test_5w,y_hat2_4_cr)

0.5

In [113]:
# Model 3_1 - 3_4, adding layers
n_steps = X_train_cr_10.shape[1]
n_features = X_train_cr_10.shape[2]

model3_1_cr = Sequential()
model3_1_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_1_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_1_cr.add(MaxPooling1D(pool_size=2))
model3_1_cr.add(Flatten())
model3_1_cr.add(Dense(8, activation='relu')) 
model3_1_cr.add(Dense(1, activation='sigmoid'))

model3_1_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_1_cr.fit(X_train_cr_10, train_5w,epochs=30,  validation_data=(X_val_cr_10, val_5w), callbacks = [early_stopping_monitor])
y_hat3_1_cr = model3_1_cr.predict(X_test_cr_10)
y_hat3_1_cr = y_hat3_1_cr > .5
metrics.accuracy_score(test_5w,y_hat3_1_cr)


Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 25ms/step - loss: 0.7186 - binary_accuracy: 0.4850 - val_loss: 0.7379 - val_binary_accuracy: 0.4173
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.7041 - binary_accuracy: 0.4850 - val_loss: 0.7220 - val_binary_accuracy: 0.4173
Epoch 3/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6972 - binary_accuracy: 0.4850 - val_loss: 0.7125 - val_binary_accuracy: 0.4173
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6933 - binary_accuracy: 0.4830 - val_loss: 0.7074 - val_binary_accuracy: 0.4173
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6909 - binary_accuracy: 0.4810 - val_loss: 0.7023 - val_binary_accuracy: 0.4029
Epoch 6/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6895 - binary_accuracy: 0.5030 - val_loss: 0.6999 - val_binary_accuracy: 0.4245
Epoch 7/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6887 - binar

0.5147058823529411

In [114]:
n_steps = X_train_cr_25.shape[1]
n_features = X_train_cr_25.shape[2]

model3_2_cr = Sequential()
model3_2_cr.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_2_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_2_cr.add(MaxPooling1D(pool_size=2))
model3_2_cr.add(Flatten())
model3_2_cr.add(Dense(25, activation='relu')) 
model3_2_cr.add(Dense(1, activation='sigmoid'))

model3_2_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_2_cr.fit(X_train_cr_25, train_5w,epochs=30,  validation_data=(X_val_cr_25, val_5w), callbacks = [early_stopping_monitor])
y_hat3_2_cr = model3_2_cr.predict(X_test_cr_25)
y_hat3_2_cr = y_hat3_2_cr > .5
metrics.accuracy_score(test_5w,y_hat3_2_cr)


Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 19ms/step - loss: 0.6936 - binary_accuracy: 0.5070 - val_loss: 0.6914 - val_binary_accuracy: 0.5036
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6915 - binary_accuracy: 0.5291 - val_loss: 0.6909 - val_binary_accuracy: 0.5683
Epoch 3/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6898 - binary_accuracy: 0.5631 - val_loss: 0.6890 - val_binary_accuracy: 0.5612
Epoch 4/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6876 - binary_accuracy: 0.5852 - val_loss: 0.6900 - val_binary_accuracy: 0.5468
Epoch 5/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6853 - binary_accuracy: 0.5952 - val_loss: 0.6886 - val_binary_accuracy: 0.5468
Epoch 6/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6827 - binary_accuracy: 0.5952 - val_loss: 0.6891 - val_binary_accuracy: 0.5468
Epoch 7/30
16/16 [==============================] - 0s 13ms/step - loss: 0.6795 - bina

0.4264705882352941

In [115]:

n_steps = X_train_cr_40.shape[1]
n_features = X_train_cr_40.shape[2]

model3_3_cr = Sequential()
model3_3_cr.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_3_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_3_cr.add(MaxPooling1D(pool_size=2))
model3_3_cr.add(Flatten())
model3_3_cr.add(Dense(25, activation='relu')) 
model3_3_cr.add(Dense(1, activation='sigmoid'))

model3_3_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_3_cr.fit(X_train_cr_40, train_5w,epochs=30,  validation_data=(X_val_cr_40, val_5w), callbacks = [early_stopping_monitor])
y_hat3_3_cr = model3_3_cr.predict(X_test_cr_40)
y_hat3_3_cr = y_hat3_3_cr > .5
metrics.accuracy_score(test_5w,y_hat3_3_cr)


Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 18ms/step - loss: 0.6957 - binary_accuracy: 0.4890 - val_loss: 0.6913 - val_binary_accuracy: 0.5036
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6876 - binary_accuracy: 0.5651 - val_loss: 0.6930 - val_binary_accuracy: 0.4964
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6831 - binary_accuracy: 0.5691 - val_loss: 0.6919 - val_binary_accuracy: 0.5324
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6795 - binary_accuracy: 0.5772 - val_loss: 0.6902 - val_binary_accuracy: 0.5324
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6747 - binary_accuracy: 0.6132 - val_loss: 0.6935 - val_binary_accuracy: 0.4892
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6700 - binary_accuracy: 0.6333 - val_loss: 0.6930 - val_binary_accuracy: 0.5036
Epoch 7/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6641 - binar

0.4852941176470588

In [116]:
n_steps = X_train_cr_55.shape[1]
n_features = X_train_cr_55.shape[2]

model3_4_cr = Sequential()
model3_4_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_4_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_4_cr.add(MaxPooling1D(pool_size=2))
model3_4_cr.add(Flatten())
model3_4_cr.add(Dense(25, activation='relu')) 
model3_4_cr.add(Dense(1, activation='sigmoid'))

model3_4_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_4_cr.fit(X_train_cr_55, train_5w,epochs=30,  validation_data=(X_val_cr_55, val_5w), callbacks = [early_stopping_monitor])
y_hat3_4_cr = model3_4_cr.predict(X_test_cr_55)
y_hat3_4_cr = y_hat3_4_cr > .5
metrics.accuracy_score(test_5w,y_hat3_4_cr)

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 17ms/step - loss: 0.6983 - binary_accuracy: 0.5090 - val_loss: 0.6894 - val_binary_accuracy: 0.5899
Epoch 2/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6943 - binary_accuracy: 0.5271 - val_loss: 0.6895 - val_binary_accuracy: 0.6043
Epoch 3/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6929 - binary_accuracy: 0.5291 - val_loss: 0.6910 - val_binary_accuracy: 0.5683
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6914 - binary_accuracy: 0.5451 - val_loss: 0.6920 - val_binary_accuracy: 0.5396
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6904 - binary_accuracy: 0.5511 - val_loss: 0.6934 - val_binary_accuracy: 0.5108


0.5147058823529411

# Changes to the Target Window

In [118]:
X_train_kb_10, X_val_kb_10, X_test_kb_10 = kbest_creator(10, X_train.drop('target_3', axis=1), X_val.drop('target_3', axis=1), X_test.drop('target_3', axis=1)) 
X_train_kb_25, X_val_kb_25, X_test_kb_25 = kbest_creator(25, X_train.drop('target_3', axis=1), X_val.drop('target_3', axis=1), X_test.drop('target_3', axis=1)) 
X_train_kb_40, X_val_kb_40, X_test_kb_40 = kbest_creator(40, X_train.drop('target_3', axis=1), X_val.drop('target_3', axis=1), X_test.drop('target_3', axis=1)) 
X_train_kb_55, X_val_kb_55, X_test_kb_55 = kbest_creator(55, X_train.drop('target_3', axis=1), X_val.drop('target_3', axis=1), X_test.drop('target_3', axis=1)) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [119]:
def df_to_X_y_mod(df, target, window_size=5):
  # converts to matrix of numpy arrays
  X = []
  y = []
  for i in range(len(df)-window_size): # length of data frame - window_size so it does't take empty values at the end, 
    # does force you to loose the last 5 values, could fix with padding
    row = [r for r in df[i:i+window_size]] # grabs row i and all rows above within the window size length
    X.append(row) # creates 3 dimentional array, (# obseravtions, # rows in window, # features)
    label = target[i+window_size-1] # pulls the target variable after the window, target varible needs to be column zero in this 
    y.append(label) # returns (N,) martix of targets i+window_length time periods away
  return np.array(X), np.array(y)
  


In [120]:
# converting to window format, in this case 5 periods
X_train_kb_40, train_5w = df_to_X_y2(X_train_kb_40,y_train)
X_val_kb_40, val_5w = df_to_X_y2(X_val_kb_40, y_val)
X_test_kb_40, test_5w = df_to_X_y2(X_test_kb_40,y_test) 

X_train_kb_10, _ = df_to_X_y2(X_train_kb_10,y_train)
X_val_kb_10, _ = df_to_X_y2(X_val_kb_10, y_val)
X_test_kb_10, _ = df_to_X_y2(X_test_kb_10,y_test) 

X_train_kb_25, _ = df_to_X_y2(X_train_kb_25,y_train)
X_val_kb_25, _ = df_to_X_y2(X_val_kb_25, y_val)
X_test_kb_25, _ = df_to_X_y2(X_test_kb_25,y_test) 

X_train_kb_55, _ = df_to_X_y2(X_train_kb_55,y_train)
X_val_kb_55, _ = df_to_X_y2(X_val_kb_55, y_val)
X_test_kb_55, _ = df_to_X_y2(X_test_kb_55,y_test) 

In [121]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model_mod4 = Sequential()
model_mod4.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model_mod4.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model_mod4.add(MaxPooling1D(pool_size=2))
model_mod4.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model_mod4.add(Flatten())
model_mod4.add(Dense(25, activation='relu')) 
model_mod4.add(Dense(1, activation='sigmoid'))

model_mod4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_mod4.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 3s 21ms/step - loss: 0.6954 - binary_accuracy: 0.4830 - val_loss: 0.6984 - val_binary_accuracy: 0.4173
Epoch 2/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6933 - binary_accuracy: 0.4930 - val_loss: 0.6945 - val_binary_accuracy: 0.4604
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6923 - binary_accuracy: 0.5170 - val_loss: 0.6937 - val_binary_accuracy: 0.5108
Epoch 4/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6915 - binary_accuracy: 0.5431 - val_loss: 0.6934 - val_binary_accuracy: 0.5252
Epoch 5/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6911 - binary_accuracy: 0.5752 - val_loss: 0.6940 - val_binary_accuracy: 0.5036
Epoch 6/30
16/16 [==============================] - 0s 10ms/step - loss: 0.6903 - binary_accuracy: 0.5772 - val_loss: 0.6915 - val_binary_accuracy: 0.5324
Epoch 7/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6895 - bin

In [123]:
y_hat_mod4_kb = model_mod4.predict(X_test_kb_25)
y_hat_mod4_kb = y_hat_mod4_kb > .5
metrics.accuracy_score(test_5w,y_hat_mod4_kb)

0.38235294117647056

In [124]:
model_mod3 = Sequential()
model_mod3.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model_mod3.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model_mod3.add(MaxPooling1D(pool_size=2))
model_mod3.add(Flatten())
model_mod3.add(Dense(25, activation='relu')) 
model_mod3.add(Dense(1, activation='sigmoid'))

model_mod3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_mod3.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 37ms/step - loss: 0.6943 - binary_accuracy: 0.4910 - val_loss: 0.7065 - val_binary_accuracy: 0.4460
Epoch 2/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6899 - binary_accuracy: 0.5491 - val_loss: 0.6984 - val_binary_accuracy: 0.4388
Epoch 3/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6872 - binary_accuracy: 0.5852 - val_loss: 0.6969 - val_binary_accuracy: 0.4532
Epoch 4/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6856 - binary_accuracy: 0.5992 - val_loss: 0.6956 - val_binary_accuracy: 0.4676
Epoch 5/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6842 - binary_accuracy: 0.5852 - val_loss: 0.6929 - val_binary_accuracy: 0.4748
Epoch 6/30
16/16 [==============================] - 0s 9ms/step - loss: 0.6831 - binary_accuracy: 0.5912 - val_loss: 0.6943 - val_binary_accuracy: 0.4820
Epoch 7/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6812 - binar

In [125]:
y_hat_mod3_kb = model_mod3.predict(X_test_kb_25)
y_hat_mod3_kb = y_hat_mod3_kb > .5
metrics.accuracy_score(test_5w,y_hat_mod3_kb)

0.47058823529411764

In [128]:
model_mod2 = Sequential()
model_mod2.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model_mod2.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model_mod2.add(Flatten())
model_mod2.add(Dense(25, activation='relu')) 
model_mod2.add(Dense(1, activation='sigmoid'))

model_mod2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_mod2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w))

Epoch 1/30
16/16 [==============================] - 1s 19ms/step - loss: 0.6991 - binary_accuracy: 0.4509 - val_loss: 0.7007 - val_binary_accuracy: 0.4245
Epoch 2/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6945 - binary_accuracy: 0.4749 - val_loss: 0.6964 - val_binary_accuracy: 0.4748
Epoch 3/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6916 - binary_accuracy: 0.5391 - val_loss: 0.6939 - val_binary_accuracy: 0.5108
Epoch 4/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6897 - binary_accuracy: 0.5591 - val_loss: 0.6911 - val_binary_accuracy: 0.5324
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6876 - binary_accuracy: 0.5611 - val_loss: 0.6909 - val_binary_accuracy: 0.5540
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6857 - binary_accuracy: 0.5651 - val_loss: 0.6905 - val_binary_accuracy: 0.5468
Epoch 7/30
16/16 [==============================] - 0s 9ms/step - loss: 0.

In [129]:
y_hat_mod2_kb = model_mod2.predict(X_test_kb_25)
y_hat_mod2_kb = y_hat_mod2_kb > .5
metrics.accuracy_score(test_5w,y_hat_mod2_kb)

0.47058823529411764

In [131]:
model_mod1 = Sequential()
model_mod1.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model_mod1.add(Flatten())
model_mod1.add(Dense(25, activation='relu')) 
model_mod1.add(Dense(1, activation='sigmoid'))

model_mod1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_mod1.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w))

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 22ms/step - loss: 0.7098 - binary_accuracy: 0.5030 - val_loss: 0.7170 - val_binary_accuracy: 0.3957
Epoch 2/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6922 - binary_accuracy: 0.5150 - val_loss: 0.7008 - val_binary_accuracy: 0.4604
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6856 - binary_accuracy: 0.5351 - val_loss: 0.6925 - val_binary_accuracy: 0.4964
Epoch 4/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6808 - binary_accuracy: 0.5591 - val_loss: 0.6899 - val_binary_accuracy: 0.5755
Epoch 5/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6769 - binary_accuracy: 0.5792 - val_loss: 0.6925 - val_binary_accuracy: 0.5540
Epoch 6/30
16/16 [==============================] - 0s 10ms/step - loss: 0.6726 - binary_accuracy: 0.5852 - val_loss: 0.6908 - val_binary_accuracy: 0.5612
Epoch 7/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6699 - bin

In [132]:
y_hat_mod1_kb = model_mod1.predict(X_test_kb_25)
y_hat_mod1_kb = y_hat_mod1_kb > .5
metrics.accuracy_score(test_5w,y_hat_mod1_kb)

0.5